In [1]:
# !pip install transformers==4.28.1

In [2]:
# !pip install datasets

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TextDataset, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
data = pd.read_csv("/kaggle/input/scrapped-lyricsfinallcsv/Scrapped_lyricsfinall.csv")


In [5]:
data["Lyrics"]=data["Lyrics"].str.replace('\n',' ')

In [6]:
data["Lyrics"]

0       انا ادري مين انا يا حبيبي مءمن ان السعي غير مص...
1       ايه يا اميره عن يدي بعيده عايشه عيشه غير العيش...
2       والموج عالي بس سباح وهعومواللي يعيش بقي يا ما ...
3       اللازمه ويجزدايره ع المصلحهكلها دايره ع المصلح...
4       عيني منها بشكل جدي وتقيله مش تقلانه اتبخر سحري...
                              ...                        
1023    اوپا اوپا اوپا اوپا ها ماشي ها ماشي اوپا اوپا ...
1024    تجيلي الصبح قبل الشغل رايق زي  بعد الضهر برفع ...
1025    منمتش من امبارح عشان شاحب قفلت التكييف قامت سا...
1026    قابلتها مره بقت my bae بتشدني جامد اكني Lion k...
1027    انا بكلم واحده تانيه وبقولك عشان اريح عشان مبق...
Name: Lyrics, Length: 1028, dtype: object

In [7]:
data["input_text"] = "artist: " + data["Artist"] + " genre: " + data["Genre"].astype(str) 

# data["input_text"] = "artist: " + data["Artist"] + " genre: " + data["Genre"].astype(str) + " lyrics: " + data['Lyrics'].str.split('\n').str[:4].str.join('\n')
data["target_text"] = data["Lyrics"]
data = data[["input_text", "target_text"]]

In [8]:
# dataset = Dataset.from_pandas(data)
# train_dataset, test_dataset = dataset.train_test_split(test_size=0.1)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)


In [9]:
train_data

,input_text,target_text
898,artist: ايهاب توفيق genre: 1,مشاعري معاك حبيبي بايه تفسرها لاول مره حاجه في...
66,artist: مروان موسي genre: 0,عصام كن انت البطل كن الكينج بضربه واحده تحدي ا...
717,artist: العسيلي genre: 1,خاينه نسيتي اللي قلنا انا وانتي عليهقلنا هنبعد...
67,artist: مروان موسي genre: 0,اه يا حبيبتي كان نفسي تكوني موجوده معانا نزل ...
260,artist: تامر حسني genre: 1,انا مقدرش ابعد ثانيهانا بعدك مليش في الدنيايا ...
...,...,...
458,artist: محمد منير genre: 0,جي من بلاد البعيده لا زاد ولا ميه وغريتي صحبتي...
330,artist: رامي صبري genre: 0,غمضت عيني وقولت نفسي اشوفها ثانيوالمس ايديها و...
466,artist: محمد منير genre: 2,يا عزيز عيني وانا بدي اروح بلدي ليله نمت فيها ...
121,artist: ابو الانوار genre: 0,كلمات اغنيه تلاته زيها ابيوسف و ابو الانوار مش...


In [10]:
train_data['input_text'][0]

'artist: ويجز genre: 4'

In [11]:
train_data['target_text'][0]

'انا ادري مين انا يا حبيبي مءمن ان السعي غير مصيري رميت والبحر يشهد بتعوزني تلاقيني بطل وقت مواجهه وشعبي مستنيني زي ما يكون بحلم تعالي شوف الشغل في السما بلمع ومش مستني الشكر وبالظبط دي كوره عشان كده بغني بقول ومن قلبي الكوره تسمعني حيوا اللي يجمعنا وما عاش اللي يفرقنا غنيها وسمعنا حيوا كل عباد الله حيوا اللي يجمعنا وما عاش اللي يفرقنا غنيها وسمعنا حيوا كل عباد الله عز العرب تعالي شوف الكرم عندي كوره وهدف عندي كوره وطرب عز العرب تعالي شوف الكرم عندي كوره وهدف عندي كوره وطرب حيوا مرحب و الشباك طارحه العرب ترعاكي يا فرحه سدد وانا اشوف اقول ما شاء الله من قطر لكل عباد الله ميجيبهاش غير ابطالها وبروس مرفوعه بحضرها كل البشر هتشرفنا كل العالم هنعرفها لا اسال شكلك ولا عنوانك مرحب بيك ناسك ومكانك الكوره لغه كل العالم ورينا كل اللي في امكانك وفيت قد ما وعدت حققت قد ما حلمت كلام كتير معمليش سقف بعد غيمه ضحكتلي الشمس ورفرف يا علمي اعلي وجدد املي انا طموح وسريع وعنيد كاس العالم عندي والرب واحد بالحب واحد ده انا وانت انسان بنسي وبسامح قالوا اختلفنا وانا قلت جايز بس فيكوا منا ولسه حبايب حيوا اللي يج

In [12]:
# !pip install sentencepiece

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("aubmindlab/aragpt2-base")
model = GPT2LMHeadModel.from_pretrained("aubmindlab/aragpt2-base")
# tokenizer = T5Tokenizer.from_pretrained("UBC-NLP/AraT5-base")
# model = T5ForConditionalGeneration.from_pretrained("UBC-NLP/AraT5-base")
tokenizer.pad_token = tokenizer.eos_token

In [14]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

In [15]:
train_dataset

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 925
})

In [16]:
test_dataset

Dataset({
    features: ['input_text', 'target_text'],
    num_rows: 103
})

In [17]:
import numpy as np

def tokenize(example):
    input_text = example["input_text"]
    target_text = example["target_text"]

    input_tokens = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    target_tokens = tokenizer(target_text, truncation=True, padding="max_length", max_length=128)

    return {
        "input_ids": np.array(input_tokens["input_ids"]),
        "attention_mask": np.array(input_tokens["attention_mask"]),
        "labels": np.array(target_tokens["input_ids"]),
    }


In [18]:
# train_dataset = train_dataset.map(tokenize, batched=True,remove_columns=["input_text","target_text"])
# test_dataset = test_dataset.map(tokenize, batched=True,remove_columns=["input_text","target_text"])
train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=["input_text", "target_text"])
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=["input_text", "target_text"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 925
})

In [20]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 103
})

In [21]:
# training_args = Seq2SeqTrainingArguments(
#     output_dir="output",
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,
#     task = "title_generation" --text_column "document" --summary_column "title"
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer)

In [26]:
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",
    save_strategy='epoch'
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.69 GiB (GPU 0; 15.90 GiB total capacity; 9.41 GiB already allocated; 2.20 GiB free; 12.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# model.save_pretrained("fine_tuned_t5")
# tokenizer.save_pretrained("fine_tuned_t5")


In [ ]:
# # model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5")
# # tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5")

# def generate_lyrics(artist, genre):
#     input_text = f"artist: {artist} genre: {genre} lyrics:"
# # input_text = f"'artist: ويجز genre: 4"
# input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda')
# output_ids = model.generate(input_ids, max_length=128)
# print(tokenizer.decode(output_ids[0]))

# generated_lyrics = generate_lyrics("ويجز", "1")
# print(generated_lyrics)

In [ ]:
# input_text = "انا ادري مين انا يا حبيبي"
input_text="artist: ويجز genre: 4"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
output = model.generate(input_ids, max_length=128)
# print(tokenizer.batch_decode(output[0]))
for i, generated_text in enumerate(tokenizer.batch_decode(output)):
    print(f"Generated text {i + 1}:")
    print(generated_text)
